# 學歷

In [ ]:
import pandas as pd
df = pd.read_csv("103-All_concat.csv")  
df

## 初步測試

### LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder( )
class_le.fit( df[ '學歷' ] )

In [ ]:
df[ '學歷' ]  = class_le.transform( df['學歷' ] )

In [ ]:
df

### Mapping

In [ ]:
class_mapping = { '研究所':6, '大學':5, '專科':4,'高中':3,'高職' :3,'國中' : 2 ,'國小':2} #最後無數字者 歸為1（國小以下）

In [ ]:
df[ '學歷' ] = df[ '學歷' ].map(class_mapping)

In [ ]:
df

## ckip_transformers  NLP Tools

In [ ]:
from ckip_transformers import __version__
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
print(__version__)
ws_driver = CkipWordSegmenter(level=3)
pos_driver = CkipPosTagger(level=3)
ner_driver = CkipNerChunker(level=3)

In [ ]:
test = '一、南華大學旅遊管理研究所畢業\n二、南開科技大學畢業\n三、南投高中畢業\n四、南投國中畢業\n五、平和國小畢業'
ws  = ws_driver(test)
pos = pos_driver(ws)
ner = ner_driver(test)
print('斷詞 ： ',ws)
print('詞性標記 ： ',pos)
print('實體辨識 ： ',ner)

In [ ]:
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        res.append(f"{word_ws}({word_pos})")
    return "\u3000".join(res)

In [ ]:
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(test, ws, pos, ner):
    print(sentence)
    print(pack_ws_pos_sentece(sentence_ws, sentence_pos))
    for entity in sentence_ner:
        print(entity)
    print()

In [ ]:
from transformers import (
  BertTokenizerFast,
  AutoModel,
)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModel.from_pretrained('ckiplab/bert-base-chinese-ws')

In [ ]:
tokens = tokenizer.tokenize(test)
ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
print(tokens)
print(ids)

## jieba

In [ ]:
import jieba
jieba.case_sensitive = True
text = '一、南華大學旅遊管理研究所畢業\n二、南開科技大學畢業\n三、南投高中畢業\n四、南投國中畢業\n五、平和國小畢業'
print('預設(精確模式) ： ')
print(jieba.lcut(text, cut_all=False, HMM=True))
print('--------------------------------------------------------------------------------------------------------------')
print('搜尋引擎模式 : ')
print(jieba.lcut_for_search(text)) # 搜尋引擎模式
# print('--------------------------------------------------------------------------------------------------------------')
# print('精確模式 : ')
# print(jieba.lcut(text, cut_all = False)) # 精確模式

In [ ]:
a = jieba.lcut(text, cut_all=False, HMM=True)
a

In [ ]:
df1 = pd.DataFrame({
        'Names': [
            'Roger Williams, Anne Graham',
            'Joe Smoe, Elliot Ezekiel',
            'Todd Roger'
         ]
    })
m = {
    'Roger Williams': 1234,
    'Joe Smoe': 898, 'Elliot Ezekiel': 8458, 'Todd Roger': 856
} #, 'Anne Graham': 4892

df1 = df1.replace(pd.Series(m).astype(str), regex=True)
df1

### My data

In [ ]:
def preprocess(question):
    words = jieba.cut(question) ## 精準模式
#     words = list(jieba.cut(question, cut_all=True)) ## 全斷詞模式
    return list(words)
df['words'] = df['學歷'].apply(preprocess)
df.head()

In [ ]:
#list to string
df['words'] = [','.join(map(str, l)) for l in df['words']]
df.head()

In [ ]:
# replace
class_mapping = { '研究所':6, '大學':5, '專科':4,'高中':3,'高職' :3,'國中' : 2 ,'國小':2} #最後無數字者 歸為1（國小以下）
df[ 'words' ] = df[ 'words' ].replace(pd.Series(class_mapping).astype(str), regex=True)

In [ ]:
df

In [ ]:
#只保留數字
df['words'] = df.words.str.extract('(\d+)') #.astype(int) df.A.str.extract('(\d+)')
df 

In [ ]:
#計算nan數量
total_nan_values = df['words'].isnull().sum().sum()
print ('nan數量 : ',total_nan_values)

## 直接文字替換
class_mapping = { '研究所':6, '大學':5, '專科':4,'高中':3,'高職' :3,'國中' : 2 ,'國小':2} #最後無數字者 歸為1（國小以下）

In [1]:
import pandas as pd
df = pd.read_csv("103-two_compare.csv") 
df

,地區,村里碼,姓名,號次,性別,出生年次,推薦政黨,得票數,得票率,當選註記,是否現任,編碼,學歷,經歷,政見
0,南投縣南投市平和里,12,謝清輝,1,男,1950,無黨籍及未經政黨推薦,1883,0.5122,1,是,370401121,南投市平和國小\n中興國中\n私立宜寧高中機工科畢業夜校,1、南投市平和里第17、18 、19屆里長\n2、中宏有限公司課長\n3、南投縣總上會上級代...,一、維持專責呈長邢秤，為呈民服務，做好呈政事務。\n二、繼續關懷弱勢族群，辦受理老人、屮低收...
1,南投縣南投市平和里,12,曾聰明,2,男,1951,無黨籍及未經政黨推薦,1793,0.4877,0,否,370401122,南投市平和國小\n陸軍第二士官學校,天明宮主任委員\n平和社區發展協會理事,熱心為里社區爭福利\n為守望相助爭福利\n認真服務里民
2,南投縣南投市三和里,16,謝銘津,1,男,1951,無黨籍及未經政黨推薦,959,0.4892,0,否,370401161,南投國小\n南投初中\n陸軍第一士校高中畢業,重興建材行。\n南投縣建材公會理事長。\n南投縣謝姓宗親會理事長。\n南投市三和社區發展協會...,一專職服務，認真工作，建設里社區。\n二燈要亮，路要平，水要通及增設防盜系統。\n三、關心婦...
3,南投縣南投市三和里,16,吳志鵬,2,男,1961,無黨籍及未經政黨推薦,1001,0.5107,1,是,370401162,南投國小\n南投國中\n國立南投高商,現任里長\n南投市農會農事小組長\n南投市農會代表\n南投市農會理事\n南投市義消分隊隊員\...,1 、推行政令、反映民意。\n2 、推動里民文康、休閒自強活動促進里民感情。\n3 、代辦各...
4,南投縣南投市嘉興里,17,張燈示,1,男,1954,無黨籍及未經政黨推薦,522,0.3546,0,否,370401171,嘉和國小\n南投高中初級部\n沙鹿高工畢業,一、南投市後備軍人輔導中心輔導員\n二、南投國際青年商會會員、OB會友。\n三、彰化汽車客運...,一、提昇里民耕作農作物、園藝、栽植管理知識，不定期聘請專業人員授課講習、期使本少利多效率·\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7403,雲林縣水林鄉大溝村,14,李聯勝,2,男,1942,無黨籍及未經政黨推薦,431,0.5561,1,是,370520142,嘉義高農肄業。,水林鄉第十七、十八、十九當選大溝村長。,服務村民為目的。
7404,雲林縣水林鄉松北村,18,陳永祥,1,男,1953,無黨籍及未經政黨推薦,564,0.7203,1,是,370520181,蔦松國小畢業。,松北村第16屆村長。松北村第19屆村長。,"一、全心全力為村民服務。二、主動認真勤快爭取地方建設。三、加強美綠化工程,改善居民居住品質及環境。"
7405,雲林縣水林鄉松北村,18,黃永成,2,男,1953,無黨籍及未經政黨推薦,219,0.2796,0,否,370520182,蔦松國小畢業。,蔦松國小42屆家長會長。,一、爭取村內農水路。二、爭取老人弱勢福利。三、爭取社區老人福利。
7406,雲林縣水林鄉溪墘村,24,蔡錦明,1,男,1951,無黨籍及未經政黨推薦,325,0.4887,0,否,370520241,和安國小。,15、16、17農會代表。,一、服務村民。二、水道路修護。三、整理環境衛生。四、關心老人與福利。五、促進社區發展。


In [2]:
#研究所
df2 = df.copy()
df2['學歷'].loc[df2['學歷'].str.contains('研究所')] = "6" #, '學歷'
#大學
df2['學歷'].loc[df2['學歷'].str.contains('大學')] = "5"
# 專科 
df2['學歷'].loc[df2['學歷'].str.contains('專科')] = "4"
#'高中':3 #'高職' :3
df2['學歷'].loc[df2['學歷'].str.contains('高中|高職')] = "3"
# '國中' : 2  '國小':2
df2['學歷'].loc[df2['學歷'].str.contains('國中|國小')] = "2"

/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


無編號繼續處理

In [3]:
#103
df2['學歷'].loc[df2['學歷'].str.contains('碩士')] = "6"
df2['學歷'].loc[df2['學歷'].str.contains('台北師範|師範學院|中正理工學院土木系畢業|淡江文理學院|學士|陸軍官校|官校|軍官學校')] = "5"
df2['學歷'].loc[df2['學歷'].str.contains('醫專|校專|工專|農工|商專|師專|技術學院|專料|農專|陸軍政工|護專|專修班|企專|體專|二專|藥專|海專|家專|行專|大專|專校|五專')] = "4"
df2['學歷'].loc[df2['學歷'].str.contains('高商|商工|高級農工|高工|工商|職業學校|高農|高級中學|商職|農校|附工|家商|高級|工家|陸軍第二士校|機械學校|海軍通信電子學校|陸軍士官學校|警察學校|新生醫校|陸軍工兵學校|藝校|幹校|女中|鳳商|二中|陸軍|三中|一中')] = "3"
df2['學歷'].loc[df2['學歷'].str.contains('國民小學|初中|初商|初級中學|國民學校|國民中學|國校|中學|初農|國民|岡中|小學|中肄業|中寮鄉龍眼林國\n民學校|港中|苓和國學|初畢|初職')] = "2"
df2['學歷'].loc[df2['學歷'].str.contains('無|學歷|民教班|識字|其他')] = "1"

df2['學歷'].loc[df2['學歷'].str.contains('學院')] = "5"
df2['學歷'].loc[df2['學歷'].str.contains('新專|漁航科|藥劑科')] = "4"
df2['學歷'].loc[df2['學歷'].str.contains('學校|佳農|雄工|兵工|工校|家職|草商|職業')] = "3"
df2['學歷'].loc[df2['學歷'].str.contains('五省中汐聯分部|東水|大同分校|初小|高小|苗中|初水')] = "2"

In [ ]:
#107
df2['學歷'].loc[df2['學歷'].str.contains('碩士|博士|所中碩山')] = "6"
df2['學歷'].loc[df2['學歷'].str.contains('台北師範|師範學院|中正理工學院土木系畢業|淡江文理學院|學士|陸軍官校|官校|軍官學校')] = "5"
df2['學歷'].loc[df2['學歷'].str.contains('醫專|校專|工專|農工|商專|師專|技術學院|專料|農專|陸軍政工|護專|專修班|企專|體專|二專|藥專|海專|家專|行專|大專|專校|五專')] = "4"
df2['學歷'].loc[df2['學歷'].str.contains('高商|商工|高級農工|高工|工商|職業學校|高農|高級中學|商職|農校|附工|家商|高級|工家|陸軍第二士校|機械學校|海軍通信電子學校|陸軍士官學校|警察學校|新生醫校|陸軍工兵學校|藝校|幹校|女中|鳳商|二中|陸軍|三中|一中')] = "3"
df2['學歷'].loc[df2['學歷'].str.contains('國民小學|初中|初商|初級中學|國民學校|國民中學|國校|中學|初農|國民|岡中|小學|中肄業|中寮鄉龍眼林國\n民學校|港中|苓和國學|初畢|初職')] = "2"
df2['學歷'].loc[df2['學歷'].str.contains('無|學歷|民教班|識字|其他|29|自修')] = "1"

df2['學歷'].loc[df2['學歷'].str.contains('學院|牧獸醫科|師大附㆗|大\n學|科技大|㆗華大|教育大|義守大|實踐大|空㆗大')] = "5"
df2['學歷'].loc[df2['學歷'].str.contains('新專|漁航科|藥劑科|行政專|商科|體\n專|農\n工|稻江科技|農 \n工|職校|陸官專|工専|商専|華岡藝術|空軍機械')] = "4"
df2['學歷'].loc[df2['學歷'].str.contains('學校|佳農|雄工|兵工|工校|家職|草商|彰工|高肄|商\n工|高\n職|高\n中|商高|北市商|高㆗|高\n工|農 工|預 備 班|高畢|海事|汽修科|士校|警察學|商畢|科|商曬|護工|職 業|職業')] = "3"
df2['學歷'].loc[df2['學歷'].str.contains('五省中汐聯分部|東水|大同分校|初小|高小|苗中|初水|鹿野酮中|中\n學|國\n中|酮小|工中|補校|國 中|國㆗|初㆗|平㆗|中 學|國 小|道明㆗|大林㆗|小|㆗|園中')] = "2"

In [4]:
df2[(df2['學歷'] != '1') & (df2['學歷'] != '2') & (df2['學歷'] != '3') & (df2['學歷'] != '4') & (df2['學歷'] != '5') & (df2['學歷'] != '6')]

,地區,村里碼,姓名,號次,性別,出生年次,推薦政黨,得票數,得票率,當選註記,是否現任,編碼,學歷,經歷,政見
2698,屏東縣竹田鄉頭崙村,5,朱達明,1,男,1953,無黨籍及未經政黨推薦,677,0.6534,1,是,370714051,一、中國造船廠員工二、竹田鄉第19屆頭崙村長,一、致力提升村民福祉，加強關懷弱勢家庭。二、服務不分姓氏族群，為村里、為頭崙鄉親不怕困難向前...,一、我用心努力爭取建設。二、我耐心長期維護環境。三、我真心照顧弱勢家庭。四、我全心全職水門公僕。
3366,彰化縣田尾鄉打簾村,13,羅東銘,1,男,1945,無黨籍及未經政黨推薦,521,0.3998,0,否,370321131,１.田尾鄉農會第八、九屆理事。２.田尾鄉鄉民代表會第１６、１７屆。,１.田尾鄉農會理事。２.田尾鄉後備軍人輔導中心秘書。３.彰化縣志願服務協會理事長。４.田尾鄉...,１.全心全力服務村民做一個專職的村長。２.關懷老人及低收入的生活起居。３.爭取一切的社會福利...
5620,臺中市霧峰區萬豐里,14,葉富田,1,男,1951,無黨籍及未經政黨推薦,721,0.3907,0,否,340026141,明德夜校肄業,一萬豐國小常務委員\n二縣議員助理\n三萬豐朝聖宮總務\n四萬豐秕區發展協會顧問,一．增進地方和諧，不分黨派，服務為先。\n二．結合社區建設，促進里內休閒產業的升級與繁榮。
6206,金門縣烏坵鄉小坵村,2,高金鳳,1,男,1938,中國國民黨,112,0.5773,1,否,380106021,長年居住烏坵鄉小坵村,一、為小坵民眾協助大小事件，盡職負責。二、發揮急難救助精神，做為民眾與鄉公所及軍方溝通橋樑，...,一、本著誠心熱忱為民服務、並發揮急難救助精神。二、做為民眾與鄉公所及軍方溝通橋樑。三、為民眾...
6207,金門縣烏坵鄉小坵村,2,莊金輝,2,男,1950,無黨籍及未經政黨推薦,82,0.4226,0,是,308106022,現任烏坵鄉小坵村村長。,一、服務第一。二、真心誠意、村民優先。三、認真打拚、爭取建設。四、關懷年長、扶助弱勢。,一、本著誠心熱忱為民服務、並發揮急難救助精神。二、做為民眾與鄉公所及軍方溝通橋樑。三、為民眾...


In [5]:
#103_1
df2['學歷'].loc[df2['學歷'].str.contains('中國造船廠員工二|田尾鄉農會第八|明德|烏坵鄉小坵')] = "1"

/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
#107_1
df2['學歷'].loc[df2['學歷'].str.contains('立志工曲|國光高裝|富添菌中|壯圍鄉古亭|泛大西洋輪船公司|二二|新英里')] = "1"
df2['學歷'][5308] = '1'

In [6]:
print('學歷六大類：')
print('研究所(6)人數：',len(df2[df2['學歷'] == "6"]))
print('大學(5)人數：',len(df2[df2['學歷'] == "5"]))
print('專科(4)人數：',len(df2[df2['學歷'] == "4"]))
print('高中,高職(3)人數：',len(df2[df2['學歷'] == "3"]))
print('國中,國小(2)人數：',len(df2[df2['學歷'] == "2"]))
print('小學以下(1)人數：',len(df2[df2['學歷'] == "1"]))
print('目前無編號人數：',len(df2[(df2['學歷'] != '1') & (df2['學歷'] != '2') & (df2['學歷'] != '3') & (df2['學歷'] != '4') & (df2['學歷'] != '5') & (df2['學歷'] != '6')]))

學歷六大類：
研究所(6)人數： 54
大學(5)人數： 525
專科(4)人數： 671
高中,高職(3)人數： 2124
國中,國小(2)人數： 3981
小學以下(1)人數： 53
目前無編號人數： 0


In [7]:
# 存檔
df2.to_csv("103_two_education.csv",index=False)

## 分類器

In [8]:
df3 = df2[~((df2['學歷'] != '1') & (df2['學歷'] != '2') & (df2['學歷'] != '3') & (df2['學歷'] != '4') & (df2['學歷'] != '5') & (df2['學歷'] != '6'))]
df3

,地區,村里碼,姓名,號次,性別,出生年次,推薦政黨,得票數,得票率,當選註記,是否現任,編碼,學歷,經歷,政見
0,南投縣南投市平和里,12,謝清輝,1,男,1950,無黨籍及未經政黨推薦,1883,0.5122,1,是,370401121,3,1、南投市平和里第17、18 、19屆里長\n2、中宏有限公司課長\n3、南投縣總上會上級代...,一、維持專責呈長邢秤，為呈民服務，做好呈政事務。\n二、繼續關懷弱勢族群，辦受理老人、屮低收...
1,南投縣南投市平和里,12,曾聰明,2,男,1951,無黨籍及未經政黨推薦,1793,0.4877,0,否,370401122,2,天明宮主任委員\n平和社區發展協會理事,熱心為里社區爭福利\n為守望相助爭福利\n認真服務里民
2,南投縣南投市三和里,16,謝銘津,1,男,1951,無黨籍及未經政黨推薦,959,0.4892,0,否,370401161,3,重興建材行。\n南投縣建材公會理事長。\n南投縣謝姓宗親會理事長。\n南投市三和社區發展協會...,一專職服務，認真工作，建設里社區。\n二燈要亮，路要平，水要通及增設防盜系統。\n三、關心婦...
3,南投縣南投市三和里,16,吳志鵬,2,男,1961,無黨籍及未經政黨推薦,1001,0.5107,1,是,370401162,2,現任里長\n南投市農會農事小組長\n南投市農會代表\n南投市農會理事\n南投市義消分隊隊員\...,1 、推行政令、反映民意。\n2 、推動里民文康、休閒自強活動促進里民感情。\n3 、代辦各...
4,南投縣南投市嘉興里,17,張燈示,1,男,1954,無黨籍及未經政黨推薦,522,0.3546,0,否,370401171,3,一、南投市後備軍人輔導中心輔導員\n二、南投國際青年商會會員、OB會友。\n三、彰化汽車客運...,一、提昇里民耕作農作物、園藝、栽植管理知識，不定期聘請專業人員授課講習、期使本少利多效率·\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7403,雲林縣水林鄉大溝村,14,李聯勝,2,男,1942,無黨籍及未經政黨推薦,431,0.5561,1,是,370520142,3,水林鄉第十七、十八、十九當選大溝村長。,服務村民為目的。
7404,雲林縣水林鄉松北村,18,陳永祥,1,男,1953,無黨籍及未經政黨推薦,564,0.7203,1,是,370520181,2,松北村第16屆村長。松北村第19屆村長。,"一、全心全力為村民服務。二、主動認真勤快爭取地方建設。三、加強美綠化工程,改善居民居住品質及環境。"
7405,雲林縣水林鄉松北村,18,黃永成,2,男,1953,無黨籍及未經政黨推薦,219,0.2796,0,否,370520182,2,蔦松國小42屆家長會長。,一、爭取村內農水路。二、爭取老人弱勢福利。三、爭取社區老人福利。
7406,雲林縣水林鄉溪墘村,24,蔡錦明,1,男,1951,無黨籍及未經政黨推薦,325,0.4887,0,否,370520241,2,15、16、17農會代表。,一、服務村民。二、水道路修護。三、整理環境衛生。四、關心老人與福利。五、促進社區發展。


In [9]:
# df3 = df.copy()
df_test = df3.groupby(["地區"]).出生年次.describe()
df3['year'] = df3['地區'].map(dict(zip(df_test.index,df_test['mean'])))
fliter1 = (df3['出生年次'] > df3["year"]) #年長 df[fliter1]
fliter2 = (df3['出生年次'] < df3["year"]) #年幼 df[fliter2]
fliter3 = (df3['出生年次'] == df3["year"]) #同年 df[fliter3]
# 男=1 女=0 
df3['性別'].replace("男",1,inplace=True)
df3['性別'].replace("女",0,inplace=True)
# 有政黨=1 無政黨=0
df3['推薦政黨'].replace(['中國國民黨', '民主進步黨','台灣團結聯盟','人民民主陣線','親民黨','中華統一促進黨','無黨團結聯盟','新黨','青年陽光黨','人民民主黨','中華文化民主黨','農民黨','樹黨','時代力量','全國人民黨','左翼聯盟','華裔和合黨','正義聯盟','台灣學習黨','中華民族致公黨','天宙和平統一家庭黨'],1,inplace=True)
df3['推薦政黨'].replace('無黨籍及未經政黨推薦',0,inplace=True)
# 現任=1 無現任=0
df3['是否現任'].replace("是",1,inplace=True)
df3['是否現任'].replace("否",0,inplace=True)
# 年長=2 年幼=1 同歲=0
df3.loc[fliter1,'年紀'] = 2
df3.loc[fliter2,'年紀'] = 1
df3.loc[fliter3,'年紀'] = 0
df3

,地區,村里碼,姓名,號次,性別,出生年次,推薦政黨,得票數,得票率,當選註記,是否現任,編碼,學歷,經歷,政見,year,年紀
0,南投縣南投市平和里,12,謝清輝,1,1,1950,0,1883,0.5122,1,1,370401121,3,1、南投市平和里第17、18 、19屆里長\n2、中宏有限公司課長\n3、南投縣總上會上級代...,一、維持專責呈長邢秤，為呈民服務，做好呈政事務。\n二、繼續關懷弱勢族群，辦受理老人、屮低收...,1950.5,1.0
1,南投縣南投市平和里,12,曾聰明,2,1,1951,0,1793,0.4877,0,0,370401122,2,天明宮主任委員\n平和社區發展協會理事,熱心為里社區爭福利\n為守望相助爭福利\n認真服務里民,1950.5,2.0
2,南投縣南投市三和里,16,謝銘津,1,1,1951,0,959,0.4892,0,0,370401161,3,重興建材行。\n南投縣建材公會理事長。\n南投縣謝姓宗親會理事長。\n南投市三和社區發展協會...,一專職服務，認真工作，建設里社區。\n二燈要亮，路要平，水要通及增設防盜系統。\n三、關心婦...,1956.0,1.0
3,南投縣南投市三和里,16,吳志鵬,2,1,1961,0,1001,0.5107,1,1,370401162,2,現任里長\n南投市農會農事小組長\n南投市農會代表\n南投市農會理事\n南投市義消分隊隊員\...,1 、推行政令、反映民意。\n2 、推動里民文康、休閒自強活動促進里民感情。\n3 、代辦各...,1956.0,2.0
4,南投縣南投市嘉興里,17,張燈示,1,1,1954,0,522,0.3546,0,0,370401171,3,一、南投市後備軍人輔導中心輔導員\n二、南投國際青年商會會員、OB會友。\n三、彰化汽車客運...,一、提昇里民耕作農作物、園藝、栽植管理知識，不定期聘請專業人員授課講習、期使本少利多效率·\...,1954.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7403,雲林縣水林鄉大溝村,14,李聯勝,2,1,1942,0,431,0.5561,1,1,370520142,3,水林鄉第十七、十八、十九當選大溝村長。,服務村民為目的。,1953.5,1.0
7404,雲林縣水林鄉松北村,18,陳永祥,1,1,1953,0,564,0.7203,1,1,370520181,2,松北村第16屆村長。松北村第19屆村長。,"一、全心全力為村民服務。二、主動認真勤快爭取地方建設。三、加強美綠化工程,改善居民居住品質及環境。",1953.0,0.0
7405,雲林縣水林鄉松北村,18,黃永成,2,1,1953,0,219,0.2796,0,0,370520182,2,蔦松國小42屆家長會長。,一、爭取村內農水路。二、爭取老人弱勢福利。三、爭取社區老人福利。,1953.0,0.0
7406,雲林縣水林鄉溪墘村,24,蔡錦明,1,1,1951,0,325,0.4887,0,0,370520241,2,15、16、17農會代表。,一、服務村民。二、水道路修護。三、整理環境衛生。四、關心老人與福利。五、促進社區發展。,1955.0,1.0


In [10]:
df3.drop(columns=['地區','村里碼','姓名','出生年次','得票數','得票率','編碼','經歷','政見','year'],inplace=True) #,'unique','學歷'
df3

,號次,性別,推薦政黨,當選註記,是否現任,學歷,年紀
0,1,1,0,1,1,3,1.0
1,2,1,0,0,0,2,2.0
2,1,1,0,0,0,3,1.0
3,2,1,0,1,1,2,2.0
4,1,1,0,0,0,3,1.0
...,...,...,...,...,...,...,...
7403,2,1,0,1,1,3,1.0
7404,1,1,0,1,1,2,0.0
7405,2,1,0,0,0,2,0.0
7406,1,1,0,0,0,2,1.0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
model = SVC()

df_ = df3[['號次','性別','推薦政黨','是否現任','學歷','年紀']] #,'學歷'
X = df_
y = df3['當選註記']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=101)
model.fit(X_train,y_train)
predictions = model.predict(X_test)

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[569 181]
 [329 403]]


              precision    recall  f1-score   support

           0       0.63      0.76      0.69       750
           1       0.69      0.55      0.61       732

    accuracy                           0.66      1482
   macro avg       0.66      0.65      0.65      1482
weighted avg       0.66      0.66      0.65      1482



### 連任捨棄作分析

In [18]:
import pandas as pd
df4 = pd.read_csv("107-two_compare.csv") 
df4_ = df4.groupby(["地區"]).是否現任.describe()
df4['unique'] = df4['地區'].map(dict(zip(df4_.index,df4_['unique']))) 
df4_ans = df4[df4['unique']==1] #1:拿掉一組有連任 2：拿掉整組皆無連任
# df4_ans

In [14]:
#103
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('研究所|碩士')] = "6"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('大學|台北師範|師範學院|中正理工學院土木系畢業|淡江文理學院|學士')] = "5"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('專科|醫專|校專|工專|農工|商專|師專|技術學院|專料|農專|軍官學校|陸軍政工|護專|專修班|企專|體專|二專|陸軍官校|藥專|海專|官校|家專|行專|大專|專校|五專')] = "4"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('高中|高職|高商|商工|高級農工|高工|工商|職業學校|高農|高級中學|商職|農校|附工|家商|高級|工家|陸軍第二士校|機械學校|海軍通信電子學校|陸軍士官學校|警察學校|新生醫校|陸軍工兵學校|藝校|幹校|女中|鳳商|二中|陸軍|初職|三中|一中')] = "3"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('國中|國小|國民小學|初中|初商|初級中學|國民學校|國民中學|國校|中學|初農|國民|岡中|小學|中肄業|中寮鄉龍眼林國\n民學校|港中|苓和國學|初畢')] = "2"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('無|學歷|民教班|識字|其他')] = "1"

df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('學院')] = "5"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('新專|漁航科|藥劑科|職業')] = "4"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('學校|佳農|雄工|兵工|工校|家職|草商')] = "3"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('五省中汐聯分部|東水|大同分校|初小|高小|苗中|初水')] = "2"

df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('中國造船廠員工二|田尾鄉農會第八|明德|烏坵鄉小坵')] = "1"

# df4_ans

/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [19]:
#107
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('研究所|碩士|博士|所中碩山')] = "6"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('大學|台北師範|師範學院|中正理工學院土木系畢業|淡江文理學院|學士')] = "5"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('專科|醫專|校專|工專|農工|商專|師專|技術學院|專料|農專|軍官學校|陸軍政工|護專|專修班|企專|體專|二專|陸軍官校|藥專|海專|官校|家專|行專|大專|專校|五專')] = "4"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('高中|高職|高商|商工|高級農工|高工|工商|職業學校|高農|高級中學|商職|農校|附工|家商|高級|工家|陸軍第二士校|機械學校|海軍通信電子學校|陸軍士官學校|警察學校|新生醫校|陸軍工兵學校|藝校|幹校|女中|鳳商|二中|陸軍|初職|三中|一中')] = "3"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('國中|國小|國民小學|初中|初商|初級中學|國民學校|國民中學|國校|中學|初農|國民|岡中|小學|中肄業|中寮鄉龍眼林國\n民學校|港中|苓和國學|初畢')] = "2"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('無|學歷|民教班|識字|其他|29|自修')] = "1"

df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('學院|牧獸醫科|師大附㆗|大\n學|科技大|㆗華大|教育大|義守大|實踐大|空㆗大')] = "5"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('新專|漁航科|藥劑科|職業|行政專|商科|體\n專|農\n工|稻江科技|職 業|農 \n工|職校|陸官專|工専|商専|華岡藝術|空軍機械')] = "4"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('學校|佳農|雄工|兵工|工校|家職|草商|彰工|高肄|商\n工|高\n職|高\n中|商高|北市商|高㆗|高\n工|農 工|預 備 班|高畢|海事|汽修科|士校|警察學|商畢|科|商曬|護工')] = "3"
df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('五省中汐聯分部|東水|大同分校|初小|高小|苗中|初水|鹿野酮中|中\n學|國\n中|酮小|工中|補校|國 中|國㆗|初㆗|平㆗|中 學|國 小|道明㆗|大林㆗|小|㆗|園中')] = "2"

df4_ans['學歷'].loc[df4_ans['學歷'].str.contains('立志工曲|國光高裝|富添菌中|壯圍鄉古亭|泛大西洋輪船公司|二二|新英里')] = "1"
df4_ans['學歷'][5308] = '1'
# df4_ans

/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_304713/2721147350.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_ans['學歷'][5308] = '1'


In [20]:
df_test = df4_ans.groupby(["地區"]).出生年次.describe()
df4_ans['year'] = df4_ans['地區'].map(dict(zip(df_test.index,df_test['mean'])))
fliter1 = (df4_ans['出生年次'] > df4_ans["year"]) #年長 df[fliter1]
fliter2 = (df4_ans['出生年次'] < df4_ans["year"]) #年幼 df[fliter2]
fliter3 = (df4_ans['出生年次'] == df4_ans["year"]) #同年 df[fliter3]
# 男=1 女=0 
df4_ans['性別'].replace("男",1,inplace=True)
df4_ans['性別'].replace("女",0,inplace=True)
# 有政黨=1 無政黨=0
df4_ans['推薦政黨'].replace(['中國國民黨', '民主進步黨','台灣團結聯盟','人民民主陣線','親民黨','中華統一促進黨','無黨團結聯盟','新黨','青年陽光黨','人民民主黨','中華文化民主黨','農民黨','樹黨','時代力量','全國人民黨','左翼聯盟','華裔和合黨','正義聯盟','台灣學習黨','中華民族致公黨','天宙和平統一家庭黨'],1,inplace=True)
df4_ans['推薦政黨'].replace('無黨籍及未經政黨推薦',0,inplace=True)
# 現任=1 無現任=0
df4_ans['是否現任'].replace("是",1,inplace=True)
df4_ans['是否現任'].replace("否",0,inplace=True)
# 年長=2 年幼=1 同歲=0
df4_ans.loc[fliter1,'年紀'] = 2
df4_ans.loc[fliter2,'年紀'] = 1
df4_ans.loc[fliter3,'年紀'] = 0
# df4_ans

/tmp/ipykernel_304713/1266628322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_ans['year'] = df4_ans['地區'].map(dict(zip(df_test.index,df_test['mean'])))
/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [21]:
df4_ans.drop(columns=['地區','村里碼','姓名','出生年次','得票數','得票率','編碼','經歷','政見','year','unique','是否現任'],inplace=True) #,'unique','學歷'
df4_ans

/home/coco31429/.local/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,號次,性別,推薦政黨,當選註記,學歷,年紀
0,1,1,1,1,5,2.0
1,2,0,1,0,4,1.0
2,1,1,0,0,6,1.0
3,2,0,1,1,4,2.0
14,1,1,0,1,4,2.0
...,...,...,...,...,...,...
7203,2,0,1,0,4,1.0
7224,1,0,0,0,3,1.0
7225,2,1,0,1,4,2.0
7244,1,1,0,0,2,1.0


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
model = SVC()

df_ = df4_ans[['號次','性別','推薦政黨','學歷','年紀']] #,'學歷'
X = df_
y = df4_ans['當選註記']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=101)
model.fit(X_train,y_train)
predictions = model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[50 32]
 [36 74]]


              precision    recall  f1-score   support

           0       0.58      0.61      0.60        82
           1       0.70      0.67      0.69       110

    accuracy                           0.65       192
   macro avg       0.64      0.64      0.64       192
weighted avg       0.65      0.65      0.65       192

